# 法律語法形式化
- version: 20250314
- spec version: 3.1
- history:
    - 20250312: 讀寫 json, 基本支援 LLM 建構 json
    - 20250313: 用 code 產生小架構的法規，增加管理功能
    - 20250314: 加入 query 的一些功能，順便測試一下資料
    - 20250319: 改善 prompt, 生成更正確，加幾個法

- env: 2504
## 起始化-物件與functions

In [ ]:
import json
from enum import Enum
import re
import os

dir_txt = "txt"
dir_json = "json"
def get_law_names_from_directory(directory_path):
    """
    從目錄中取得所有法規名稱的列表
    :param directory_path: 目錄的路徑
    :return: 法規名稱的列表
    """
    law_names = set()
    # Check if directory exists
    if not os.path.exists(directory_path):
        print(f"Directory not found: {directory_path}")
        return list(law_names)
    for filename in os.listdir(directory_path):
        if filename.endswith("_law_regulation.json"):
            law_name = filename.split("_law_regulation")[0]
            law_names.add(law_name)
    return list(law_names)
def handle_regex(regex,file_path,type="col2"): 
    """
    return lines
    """
    print(f"parse file_path:{file_path}")
    with open(file_path, 'r', encoding='utf-8') as file:
            test_str = file.read()
    #print(test_str)
    matches = re.finditer(regex, test_str, re.DOTALL) #re.MULTILINE
    lines = []
    if type=="col2": #regex="\*\*Q：\*\*(.*)\n\*\*A：\*\*(.*)\n"
        for matchNum, match in enumerate(matches, start=1):
            for groupNum in range(0, len(match.groups())):
                groupNum = groupNum + 1
                mark = "Q" if groupNum==1 else "A"
                group = match.group(groupNum).replace("*","")
                print_str = f"{mark}:{group}"
                print(print_str ) 
                lines.append(print_str)
    if type=="col1":
        
        for matchNum, match in enumerate(matches, start=1):
            
            for groupNum in range(0, len(match.groups())):
                groupNum = groupNum + 1
                group = match.group(groupNum)
                print_str = f"{group}"
                #print(print_str ) 
                lines.append(print_str)
    return lines    

# Helper function: Load JSON data from file
def load_json_data(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"JSON format error: {filepath}")
        return None
    
class ConceptCategory(Enum):
    """Legal concept categories enumeration."""
    CORE_CONCEPT_DEFINITION = "核心概念 - 定義" # You can keep Chinese values for display if needed
    # ... 其他類別 (other categories)

class LawMetadata:
    """
    Law MetaData object, used to integrate different types of law information.
    """
    def __init__(self, law_name = None, law_regulation=None, legal_concepts=None, hierarchy_relations=None, law_relations=None, law_articles=None):
        """
        Initializes LawMetaData object.

        Args:
            law_regulation (dict, optional): Law regulation metadata, expected as a single dict. Defaults to None.
            # ... other args
        """
        self.law_regulation = law_regulation or {}
        self.legal_concepts = legal_concepts or []
        self.hierarchy_relations = hierarchy_relations or []
        self.law_relations = law_relations or []
        self.law_articles = law_articles or []

        # PCode handling: Ensure '代號' is the PCode
        pcode_val = None
        if self.law_regulation:
            pcode_keys = ['PCode', 'pcode'] # Case-sensitive check for explicit PCode
            for p_key in pcode_keys:
                if p_key in self.law_regulation:
                    pcode_val = self.law_regulation[p_key]
                    break
            current_daihao = self.law_regulation.get('代號')
            if pcode_val and current_daihao != pcode_val:
                # Warning: Law regulation '代號' might be overwritten by explicit PCode if they differ.
                # This print might occur before self.law_name is fully determined if '法規名稱' is also missing.
                # print(f"Warning: Law regulation '代號' ({current_daihao}) differs from explicit PCode ({pcode_val}). Using explicit PCode for '代號'. Law: {self.law_regulation.get('法規名稱')}")
                self.law_regulation['代號'] = pcode_val
            elif not current_daihao and pcode_val: # '代號' is missing, but PCode is found
                self.law_regulation['代號'] = pcode_val
            # If pcode_val is None, '代號' (if it exists) is assumed to be the pcode.

        self.law_name = law_name or self.law_regulation.get("法規名稱") 
        if not self.law_name and self.law_regulation.get('代號'):
             # Fallback for law_name if '法規名稱' is missing but '代號' (PCode) exists
             self.law_name = self.law_regulation.get('代號') 
        elif not self.law_name:
             self.law_name = "default_law_name_if_all_else_fails"
        
        self.short_name = self.law_name

    @classmethod
    def from_json_files(cls, law_regulation, legal_concepts, hierarchy_relations, law_relations, law_articles):
        """
        Reads law MetaData from separate JSON files.
        """
        law_regulation_data = load_json_data(law_regulation)
        # PCode handling for data loaded from JSON
        if law_regulation_data:
            pcode_json_val = None
            pcode_json_keys = ['PCode', 'pcode']
            for p_key in pcode_json_keys:
                if p_key in law_regulation_data:
                    pcode_json_val = law_regulation_data[p_key]
                    break
            current_daihao_json = law_regulation_data.get('代號')
            law_name_for_warning = law_regulation_data.get('法規名稱', law_regulation.split('/')[-1].split('_law_regulation.json')[0])
            if pcode_json_val:
                if current_daihao_json != pcode_json_val:
                    print(f"Warning: '代號' ({current_daihao_json}) in JSON differs from PCode ({pcode_json_val}). Setting '代號' to PCode. Law: {law_name_for_warning}")
                    law_regulation_data['代號'] = pcode_json_val
            elif not current_daihao_json:
                print(f"Warning: PCode ('PCode' or 'pcode') or '代號' not found in {law_regulation} for {law_name_for_warning}. DB operations might fail if '代號' is not later set to a valid PCode.")
        
        legal_concepts_data = load_json_data(legal_concepts)
        hierarchy_relations_data = load_json_data(hierarchy_relations)
        law_relations_data = load_json_data(law_relations)
        law_articles_data = load_json_data(law_articles)

        if not all([law_regulation_data, law_articles_data]):
            print(f"Error: Core data (regulation or articles) missing for one of the paths provided.")
            print(f"Paths: LR:{law_regulation}, LC:{legal_concepts}, LH:{hierarchy_relations}, LRel:{law_relations}, LA:{law_articles}")
            return None
        
        if legal_concepts_data is None: legal_concepts_data = []
        if hierarchy_relations_data is None: hierarchy_relations_data = []
        if law_relations_data is None: law_relations_data = []

        return cls(
            law_regulation=law_regulation_data,
            legal_concepts=legal_concepts_data,
            hierarchy_relations=hierarchy_relations_data,
            law_relations=law_relations_data,
            law_articles=law_articles_data
        )
    

    def to_json_files(self, output_prefix="gpa"):
        """
        Exports LawMetaData object to separate JSON files with law name prefix.
        """
        if not output_prefix:
            output_prefix = self.law_name 

        filepaths = {
            "law_regulation": f"{output_prefix}_law_regulation.json",
            "legal_concepts": f"{output_prefix}_legal_concepts.json",
            "hierarchy_relations": f"{output_prefix}_hierarchy_relations.json",
            "law_relations": f"{output_prefix}_law_relations.json",
            "law_articles": f"{output_prefix}_law_articles.json"
        }

        data_to_export = {
            "law_regulation": self.law_regulation,
            "legal_concepts": [
                {**concept, "概念類別": concept["概念類別"].value if isinstance(concept.get("概念類別"), ConceptCategory) else concept.get("概念類別")}
                for concept in self.legal_concepts
            ],
            "hierarchy_relations": self.hierarchy_relations,
            "law_relations": self.law_relations,
            "law_articles": self.law_articles
        }

        for key, filepath in filepaths.items():
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(data_to_export[key], f, indent=2, ensure_ascii=False)
        print(f"Exported all components for '{self.law_name}' with prefix '{output_prefix}'.")

    def renew_id(self):
        # law_id = f"LT_{self.law_name}" # This format is for the old 'code' field.
        # self.law_regulation["代號"]=law_id # '代號' should now be PCode, not overwritten here.
        # PCode is sourced from JSON or __init__; renew_id should not overwrite it with LT_lawname format.
        
        # Ensure self.law_name is based on '法規名稱' for consistent ID generation below
        # self.law_name is already set in __init__ based on 法規名稱 or PCode as fallback.

        for article in self.law_articles:
            article_code_current = article.get("代號", "")
            if not article_code_current.startswith("LA_"):
                article_id = f"LA_{self.law_name}_{article.get('條號', 'UnknownArticle')}"
                article["代號"]=article_id
        for concept in self.legal_concepts:
            concept_code_current = concept.get("代號", "")
            if not concept_code_current.startswith("LC_"):
                concept_id = f"LC_{self.law_name}_{concept.get('詞彙名稱', 'UnknownConcept')}"
                concept['代號']=concept_id
        seq = 1
        for relation in self.law_relations:
            relation_code_current = relation.get("代號", "")
            if not relation_code_current.startswith("LR_"):
                relation_id = f"LR_{self.law_name}_{seq}"
                relation['代號']=relation_id
            seq += 1
        seq_hier = 1 
        for hierarchy in self.hierarchy_relations:
            hier_code_current = hierarchy.get("關係代號", "")
            if not hier_code_current.startswith("LH_"):
                hierarchy_id = f"LH_{self.law_name}_{hierarchy.get('關聯法規', 'UnknownRelatedLaw')}_{seq_hier}"
                hierarchy['關係代號']=hierarchy_id
            seq_hier +=1
        # Use PCode for this message if available, otherwise law_name
        identifier_for_message = self.law_regulation.get('代號', self.law_name)
        print(f"IDs renewed for LawMetadata: {identifier_for_message}")


    def __repr__(self):
        return f"LawMetadata(law_name='{self.law_regulation.get('法規名稱', 'N/A')}', concept_count={len(self.legal_concepts)}, ...)"

class LawMetadataMgr:
    def __init__(self, db_conn=None):
        self.lms = {}
        self.short_names = {}
        self.dir_json = "json"
        self.db_conn = db_conn
        if self.db_conn:
            print("LawMetadataMgr initialized with a database connection.")
        else:
            print("LawMetadataMgr initialized without a database connection (JSON mode only).")

    def add_lm(self, lm , short_name=None):
        if not lm or not lm.law_name or not lm.law_regulation.get('法規名稱'):
            # Check if PCode ('代號') exists as a fallback for identifying the law regulation
            if not lm or not lm.law_regulation or not lm.law_regulation.get('代號'):
                 print("Error: Cannot add invalid or incomplete LawMetadata object (missing 法規名稱 and 代號).")
                 return
            elif not lm.law_name:
                 # If law_name is missing but PCode exists, try to use PCode as name for manager key
                 lm.law_name = lm.law_regulation.get('代號')
                 if not lm.law_name: # Still no identifier
                    print("Error: Cannot add invalid or incomplete LawMetadata object (PCode also missing).")
                    return
        
        actual_short_name = short_name if short_name else lm.law_name
        lm.short_name = actual_short_name
        
        self.short_names[actual_short_name] = lm.law_name 
        self.lms[lm.law_name] = lm
        print(f"Added '{lm.law_name}' (short: '{actual_short_name}') to in-memory manager.")

        if self.db_conn:
            # Use PCode ('代號') for DB operations if available
            pcode_for_db = lm.law_regulation.get('代號', lm.law_name) 
            print(f"DB connection found. Attempting to upsert LawMetadata for '{pcode_for_db}' to database.")
            try:
                success = upsert_law_metadata_to_db(lm, self.db_conn)
                if success:
                    print(f"Successfully upserted '{pcode_for_db}' to database.")
                else:
                    print(f"Warning: Failed to upsert '{pcode_for_db}' to database. It remains in memory only.")
            except Exception as e:
                print(f"Error during DB upsert for '{pcode_for_db}': {e}. It remains in memory only.")

    def remove_lm(self, law_name_or_short_name):
        law_name_to_remove = self.short_names.get(law_name_or_short_name, law_name_or_short_name)
        
        if law_name_to_remove in self.lms:
            lm_to_remove = self.lms[law_name_to_remove]
            # Use PCode ('代號') for deletion if available
            pcode_to_delete = lm_to_remove.law_regulation.get('代號', law_name_to_remove)
            
            del self.lms[law_name_to_remove]
            # Clean up short_names mapping
            keys_to_del_from_short_names = [k for k, v in self.short_names.items() if v == law_name_to_remove]
            for k in keys_to_del_from_short_names:
                del self.short_names[k]

            print(f"Removed '{law_name_to_remove}' (PCode: {pcode_to_delete}) from in-memory manager.")

            if self.db_conn:
                if pcode_to_delete:
                    print(f"DB connection found. Attempting to delete LawMetadata for pcode '{pcode_to_delete}' from database.")
                    try:
                        delete_law_metadata_from_db(pcode_to_delete, self.db_conn)
                    except Exception as e:
                        print(f"Error during DB delete for pcode '{pcode_to_delete}': {e}.")
                else:
                    print(f"Warning: Cannot delete '{law_name_to_remove}' from DB as its PCode ('代號') is missing.")
        else:
            print(f"Law '{law_name_or_short_name}' not found in manager.")

    def find_lm(self, law_name_or_short_name):
        law_name_to_find = self.short_names.get(law_name_or_short_name, law_name_or_short_name)
        return self.lms.get(law_name_to_find, None)

    def load_lm_bynames_from_json(self, short_names_list):
        """Loads LawMetadata from JSON files based on a list of short names."""
        loaded_count = 0
        for short_name in short_names_list:
            base_path = os.path.join(self.dir_json, short_name)
            lm = LawMetadata.from_json_files(
                f"{base_path}_law_regulation.json",
                f"{base_path}_legal_concepts.json",
                f"{base_path}_hierarchy_relations.json",
                f"{base_path}_law_relations.json",
                f"{base_path}_law_articles.json"
            )
            if lm:
                self.add_lm(lm, short_name)
                loaded_count += 1
        print(f"Finished loading from JSON. {loaded_count} laws processed.")
        
    def load_lm_from_db(self, law_pcode_to_load: str = None):
        """Loads LawMetadata from the database for a specific pcode or all laws."""
        if not self.db_conn:
            print("Error: Database connection not available for load_lm_from_db.")
            return

        laws_query = "SELECT id AS law_db_id, pcode, xml_law_name, law_metadata FROM laws" 
        params = []
        if law_pcode_to_load:
            laws_query += " WHERE pcode = %s"
            params.append(law_pcode_to_load)
        
        base_law_records = fetch_query(self.db_conn, laws_query, tuple(params) if params else None)
        if not base_law_records:
            print(f"No laws found in DB for pcode '{law_pcode_to_load if law_pcode_to_load else 'ALL'}'.")
            return

        loaded_count = 0
        if not law_pcode_to_load:
            print("Clearing existing in-memory laws before loading all from DB.")
            self.lms.clear()
            self.short_names.clear()
            
        for law_record in base_law_records:
            law_db_id, pcode, xml_law_name, law_metadata_json = law_record

            law_regulation_data = {}
            if law_metadata_json:
                law_regulation_data = json.loads(law_metadata_json) if isinstance(law_metadata_json, str) else law_metadata_json
            else:
                law_regulation_data['法規名稱'] = xml_law_name
                law_regulation_data['代號'] = pcode
            
            current_law_name = law_regulation_data.get('法規名稱', xml_law_name or pcode)
            print(f"Processing law from DB: {current_law_name} (PCode: {pcode}, DB ID: {law_db_id})")

            articles_query = "SELECT id AS article_db_id, xml_article_number, xml_chapter_section, xml_article_content, article_metadata FROM articles WHERE law_id = %s ORDER BY xml_article_number;"
            article_records = fetch_query(self.db_conn, articles_query, (law_db_id,))
            
            fetched_articles_list = []
            for art_rec in (article_records or []):
                article_db_id, xml_article_number, xml_chapter_section, xml_article_content, article_metadata_json = art_rec
                article_data = {}
                if article_metadata_json:
                    article_data = json.loads(article_metadata_json) if isinstance(article_metadata_json, str) else article_metadata_json
                else:
                    article_data['條號'] = xml_article_number
                    article_data['編章節'] = xml_chapter_section
                    article_data['條文內容'] = xml_article_content
                article_data['db_id'] = article_db_id
                fetched_articles_list.append(article_data)

            concepts_query = "SELECT data FROM legal_concepts WHERE law_id = %s;"
            concept_records = fetch_query(self.db_conn, concepts_query, (law_db_id,))
            fetched_concepts_list = [json.loads(rec[0]) if isinstance(rec[0], str) else rec[0] for rec in concept_records or [] if rec[0]]

            hierarchy_query = "SELECT data FROM law_hierarchy_relationships WHERE main_law_id = %s OR related_law_id = %s;"
            hierarchy_records = fetch_query(self.db_conn, hierarchy_query, (law_db_id, law_db_id))
            fetched_hierarchy_list = [json.loads(rec[0]) if isinstance(rec[0], str) else rec[0] for rec in hierarchy_records or [] if rec[0]]

            law_relations_query = "SELECT data FROM law_relationships WHERE main_law_id = %s OR related_law_id = %s OR main_article_id IN (SELECT id FROM articles WHERE law_id = %s) OR related_article_id IN (SELECT id FROM articles WHERE law_id = %s);"
            law_relation_records = fetch_query(self.db_conn, law_relations_query, (law_db_id, law_db_id, law_db_id, law_db_id))
            fetched_law_relations_list = [json.loads(rec[0]) if isinstance(rec[0], str) else rec[0] for rec in law_relation_records or [] if rec[0]]

            lm = LawMetadata(
                law_regulation=law_regulation_data, 
                legal_concepts=fetched_concepts_list, 
                hierarchy_relations=fetched_hierarchy_list, 
                law_relations=fetched_law_relations_list, 
                law_articles=fetched_articles_list
            )
            self.lms[lm.law_name] = lm 
            self.short_names[lm.law_name] = lm.law_name 
            print(f"  Reconstructed and added LawMetadata for '{lm.law_name}' (PCode: {pcode}) to manager.")
            loaded_count += 1
        print(f"Finished loading from DB. {loaded_count} laws processed and added to manager.")

    def export_all_to_json(self):
        """Exports all LawMetadata objects in the manager to JSON files using their short_name as prefix."""
        if not os.path.exists(self.dir_json):
            os.makedirs(self.dir_json)
            print(f"Created directory: {self.dir_json}")
        
        exported_count = 0
        for law_name, lm in self.lms.items():
            safe_short_name = lm.short_name.replace('/', '_').replace('\\', '_')
            file_prefix = os.path.join(self.dir_json, safe_short_name)
            lm.to_json_files(output_prefix=file_prefix)
            exported_count +=1
        print(f"Exported {exported_count} laws to JSON files in '{self.dir_json}' directory.")

    def __repr__(self):
        return f"LawMetadataManager(law_count={len(self.lms)}, db_connected={'Yes' if self.db_conn else 'No'})"




## Interacting with the PostgreSQL Database
This notebook has been enhanced to support PostgreSQL for persistent storage and management of `LawMetadata` objects. This offers several advantages over using only local JSON files, including:
- **Persistent Storage:** Data saved to the database will persist across notebook sessions.
- **Structured Querying:** Although the current analysis tools in this notebook are Python-based, storing data in a relational database allows for powerful SQL querying capabilities externally (e.g., for complex data retrieval, reporting, or integration with other systems).
- **Data Sharing & Centralization:** A database can serve as a central repository for legal metadata, accessible by multiple users or processes (with appropriate database permissions).

**Dual-Mode Capability:**
The notebook, particularly the `LawMetadataMgr` class, is designed to operate in a "dual mode":
1.  **JSON-Only Mode:** If a database connection is not provided to `LawMetadataMgr` during its initialization, it will work exclusively with local JSON files (loading from and saving to the `./json` directory).
2.  **Database-Integrated Mode:** If a database connection object is provided, `LawMetadataMgr` methods like `add_lm` and `remove_lm` will automatically attempt to synchronize changes with the database (i.e., upserting or deleting corresponding records).

This section provides a comprehensive guide to setting up the database, creating the schema, and performing various operations such as loading data from JSON to the DB, retrieving data from the DB into `LawMetadata` objects, deleting data, and using the analysis tools with DB-loaded data.

### A. Database Connection and Core Utility Functions
This cell defines essential functions for database interaction. It's crucial for any subsequent database operations.

**Key Functions Defined Here:**
- `get_db_connection()`: Establishes a connection to the PostgreSQL database.
  - **Configuration Required:** The connection parameters (`DB_NAME`, `DB_USER`, `DB_PASSWORD`, `DB_HOST`, `DB_PORT`) are defined at the beginning of this code cell. 
  - **Best Practice:** Set these using environment variables for security and flexibility (e.g., `os.getenv('DB_USER', 'your_default_user')`). 
  - **Manual Setup:** If not using environment variables, **you must replace the placeholder default values** (like `'your_default_db_name'`) with your actual database credentials and details.
  - **Server Accessibility:** Ensure your PostgreSQL server is running and accessible from the environment where this notebook is executed.
- `execute_query()`: Executes general SQL commands that modify the database (e.g., INSERT, UPDATE, DELETE, DDL like `CREATE TABLE`). It handles transaction commits and rollbacks.
- `fetch_query()`: Executes SELECT queries and fetches results.
- `create_db_schema()`: Reads and executes SQL from `law_db_law.sql` to set up the database tables.
- Helper ID Lookup Functions (e.g., `_get_law_db_id_by_code`): Internal functions used by the main upsert/delete operations to find database primary keys.
- `upsert_law_metadata_to_db()`: A comprehensive function to save or update a `LawMetadata` object and all its constituent parts (regulation, articles, concepts, relations) into the database.
- `delete_law_metadata_from_db()`: Deletes a law and all its associated data from the database based on its unique code.

**Prerequisites for running this cell and subsequent DB operations:**
- `psycopg2-binary` library must be installed (`pip install psycopg2-binary`).
- A PostgreSQL server must be running and accessible.
- Database connection parameters below must be correctly configured.

In [ ]:
# Install psycopg2-binary if you haven't: pip install psycopg2-binary
import psycopg2
import os
import json # Ensure json is imported here for json.dumps

# --- Database Connection Parameters --- 
# IMPORTANT: Configure these for your PostgreSQL instance.
# Using environment variables is recommended for sensitive data like passwords.
# Example: os.getenv('DB_NAME', 'your_default_db_name')
# If not using environment variables, replace the default string values directly.
DB_NAME = os.getenv('DB_NAME', 'your_default_db_name')         # Your database name
DB_USER = os.getenv('DB_USER', 'your_default_user')           # Your PostgreSQL username
DB_PASSWORD = os.getenv('DB_PASSWORD', 'your_default_password') # Your PostgreSQL password
DB_HOST = os.getenv('DB_HOST', 'localhost')                   # Database host (e.g., 'localhost' or an IP address)
DB_PORT = os.getenv('DB_PORT', '5432')                      # Database port (default for PostgreSQL is '5432')
# Ensure the PostgreSQL server is running and accessible with these credentials.

def get_db_connection():
    """Establishes a connection to the PostgreSQL database."""
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        print("Successfully connected to PostgreSQL.")
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to PostgreSQL: {e}")
        return None

def execute_query(conn, query, params=None):
    """
    Executes a query that modifies the database (INSERT, UPDATE, DELETE).
    Also handles execution of multi-statement SQL scripts.
    
    Args:
        conn: Active database connection.
        query (str): SQL query string or script.
        params (tuple, optional): Parameters for the query. Not typically used for multi-statement scripts.
    """
    cursor = None
    try:
        cursor = conn.cursor()
        cursor.execute(query, params)
        conn.commit()
        # print("Query executed successfully.") # Making this less verbose for batch operations
    except psycopg2.Error as e:
        if conn:
            conn.rollback()
        print(f"Error executing query: {e}")
        # raise  # Optionally re-raise the exception
    finally:
        if cursor:
            cursor.close()

def fetch_query(conn, query, params=None):
    """
    Executes a query and fetches results (SELECT).
    
    Args:
        conn: Active database connection.
        query (str): SQL query string.
        params (tuple, optional): Parameters for the query. Defaults to None.
        
    Returns:
        list: A list of tuples representing the fetched rows, or None if an error occurs.
    """
    cursor = None
    try:
        cursor = conn.cursor()
        cursor.execute(query, params)
        results = cursor.fetchall()
        return results
    except psycopg2.Error as e:
        print(f"Error fetching query: {e}")
        return None
    finally:
        if cursor:
            cursor.close()

def create_db_schema(conn):
    """
    Reads SQL statements from 'law_meta_db_v0.2.sql' and executes them to create the DB schema.

    Args:
        conn: Active database connection.
    """
    if 1:
        return
    try:
        with open('law_meta_db_v0.2.sql', 'r', encoding='utf-8') as f:
            sql_script = f.read()
        
        print("Attempting to create database schema from law_meta_db_v0.2.sql...")
        execute_query(conn, sql_script) 
        print("Database schema creation process completed.")
        print("Please check for any errors above from execute_query. If 'Query executed successfully.' was printed by execute_query, the script likely ran.")
        
    except FileNotFoundError:
        print("Error: law_meta_db_v0.2.sql not found. Cannot create database schema.")
    except psycopg2.Error as e:
        print(f"Error during schema creation: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        if conn:
            conn.rollback()

### --- Helper functions for DB ID lookups --- 
def _get_law_pcode_by_name(law_name, conn):
    """Fetches the pcode of a law by its name (xml_law_name)."""
    if not law_name: return None
    query = "SELECT pcode FROM laws WHERE xml_law_name = %s;"
    result = fetch_query(conn, query, (law_name,))
    return result[0][0] if result else None

def _get_law_db_id_by_pcode(pcode, conn):
    """Fetches the database ID of a law by its pcode."""
    if not pcode: return None
    query = "SELECT id FROM laws WHERE pcode = %s;"
    result = fetch_query(conn, query, (pcode,))
    return result[0][0] if result else None

def chinese_to_arabic(cn_str):
    """
    A simple converter for Chinese numeral strings found in law article numbers.
    Handles numbers up to 9999.
    e.g., "四" -> 4, "二十一" -> 21, "一百二十一" -> 121
    """
    if not isinstance(cn_str, str):
        return None

    # If it's already arabic, just return it as int
    if cn_str.isdigit():
        try:
            return int(cn_str)
        except ValueError:
            return None

    cn_map = {'零': 0, '一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9}
    unit_map = {'十': 10, '百': 100, '千': 1000}
    
    cn_str = cn_str.strip()
    # Handle "十" at the beginning, e.g., "十" -> 10, "十一" -> 11
    if cn_str.startswith('十'):
        cn_str = '一' + cn_str

    total = 0
    temp_num = 0
    for char in cn_str:
        if char in cn_map:
            temp_num = cn_map[char]
        elif char in unit_map:
            if temp_num == 0:
                temp_num = 1
            total += temp_num * unit_map[char]
            temp_num = 0
        else:
            return None # Invalid character
    total += temp_num
    return total

   

def normalize_article_number(article_str):
    """
    Converts an article number string (e.g., "第四條", "第 4 條", "第4條之1", "第三條之一")
    into a canonical format (e.g., "第 4 條", "第 4-1 條", "第 3-1 條").
    """
    if not article_str:
        return None
    
    s = article_str.strip().replace(' ', '')
    s = s.replace('条', '條') 
    
    # Handle sub-articles like "之"
    parts = s.split('之')
    
    arabic_parts = []
    for part in parts:
        # clean up each part
        p = part
        if p.startswith('第'):
            p = p[1:]
        if p.endswith('條'):
            p = p[:-1]
            
        # Convert Chinese part to Arabic
        arabic_val = chinese_to_arabic(p)
        if arabic_val is not None:
            arabic_parts.append(str(arabic_val))
        else:
            # Could not convert, maybe it's already mixed or invalid.
            # Fallback to original part, but cleaned.
            arabic_parts.append(p)
            
    numeric_part = "-".join(arabic_parts)
    return f"第 {numeric_part} 條"

def _get_article_db_id_by_law_db_id_and_article_number(law_db_id, article_number, conn):
    """
    Fetches the database ID of an article by its law_db_id and article_number.
    This version handles conversion between Chinese and Arabic numerals and varying whitespace
    by fetching all articles for the law and normalizing them in Python.
    Note: This is more robust but less efficient if called in a loop for the same law,
    as it fetches all articles for that law on each call.
    """
    if not law_db_id or not article_number:
        return None

    # 1. Normalize the target article number from the JSON
    normalized_target = normalize_article_number(article_number)
    if not normalized_target:
        return None

    # 2. Fetch all articles for the given law_db_id
    query = "SELECT id, xml_article_number FROM articles WHERE law_id = %s;"
    records = fetch_query(conn, query, (law_db_id,))
    if not records:
        return None

    # 3. Normalize each article number from the DB and find a match
    for db_id, db_article_number in records:
        if db_article_number:
            normalized_db_num = normalize_article_number(db_article_number)
            if normalized_db_num == normalized_target:
                return db_id
            
    return None

def _get_concept_db_id_by_code(concept_code, conn):
    """Fetches the database ID of a legal concept by its code."""
    if not concept_code: return None
    query = "SELECT id FROM legal_concepts WHERE code = %s;"
    result = fetch_query(conn, query, (concept_code,))
    return result[0][0] if result else None

### --- Main DB Upsert/Delete Functions --- 
def delete_law_metadata_from_db(pcode: str, conn):
    """Deletes a law and its related data from the database based on pcode."""
    if not pcode:
        print("Error: Law pcode cannot be empty for deletion.")
        return False
    print(f"Attempting to delete law with pcode: {pcode} and its related data...")
    # ON DELETE CASCADE in the schema will handle deletion of related data in other tables.
    query = "DELETE FROM laws WHERE pcode = %s;"
    execute_query(conn, query, (pcode,))
    print(f"Deletion process for law pcode {pcode} initiated.")
    return True

def upsert_law_metadata_to_db(lm: LawMetadata, conn):
    """
    Upserts a LawMetadata object into the database.
    This version focuses on updating metadata for existing laws and articles.
    - It updates the law's metadata (in 'laws' table) and each article's metadata (in 'articles' table).
    - It does NOT create or delete article records, assuming they are managed by another process (e.g., law_proc.ipynb).
    - It performs a "clean and refresh" for associated metadata: concepts are upserted, and all relationships 
      (law-to-law, law-to-article, article-to-concept) for the law are deleted and re-inserted from the LawMetadata object.
    """
    law_name = lm.law_regulation.get('法規名稱')
    if not law_name:
        print("Error: Law '法規名稱' is missing from law_regulation. Cannot upsert.")
        return False

    # 1. Verify law exists and get its pcode and db_id
    pcode = _get_law_pcode_by_name(law_name, conn)
    if not pcode:
        print(f"Error: Law with name '{law_name}' not found in the database. Upsert aborted. Please ensure the base law record exists.")
        return False
    
    law_db_id = _get_law_db_id_by_pcode(pcode, conn)
    if not law_db_id:
        print(f"Error: Law with pcode '{pcode}' found but could not retrieve its database ID. Aborting.")
        return False

    # Update the in-memory object with the correct pcode from the DB
    lm.law_regulation['代號'] = pcode
    
    print(f"Starting upsert process for law: {law_name} (PCode: {pcode}, DB ID: {law_db_id})")

    try:
        # 2. Clean up old metadata relationships for this law before inserting new data
        print(f"  Cleaning up existing relationships for law ID: {law_db_id}...")
        
        # Clean up article-to-concept links for the entire law
        execute_query(conn, "DELETE FROM article_legal_concept WHERE article_id IN (SELECT id FROM articles WHERE law_id = %s);", (law_db_id,))
        
        # Clean up law_relationships involving this law (law-law, and any links to its articles)
        execute_query(conn, """
            DELETE FROM law_relationships 
            WHERE (main_law_id = %s) OR (related_law_id = %s)
        """, (law_db_id, law_db_id))

        # Clean up law-to-law hierarchy relationships
        execute_query(conn, "DELETE FROM law_hierarchy_relationships WHERE main_law_id = %s OR related_law_id = %s;", (law_db_id, law_db_id))
        
        print("  Cleanup of relationships complete.")

        # 3. Update the main law record with the new metadata JSON
        law_metadata_json = json.dumps(lm.law_regulation)
        law_update_query = "UPDATE laws SET law_metadata = %s WHERE id = %s;"
        execute_query(conn, law_update_query, (law_metadata_json, law_db_id))
        print(f"  Updated 'law_metadata' in 'laws' table for ID: {law_db_id}")

        concept_code_to_db_id_map = {}

        # 4. Update article metadata (DO NOT INSERT/DELETE ARTICLES)
        print(f"  Updating article metadata...")
        updated_article_count = 0
        for article in lm.law_articles:
            article_number = article.get('條號')
            if not article_number:
                print("  Warning: Skipping article in JSON with missing '條號'.")
                continue

            # Find article_db_id using flexible helper
            article_db_id = _get_article_db_id_by_law_db_id_and_article_number(law_db_id, article_number, conn)

            if article_db_id:
                article_metadata_json = json.dumps(article)
                article_update_query = "UPDATE articles SET article_metadata = %s WHERE id = %s;"
                execute_query(conn, article_update_query, (article_metadata_json, article_db_id))
                article['db_id'] = article_db_id  # Store db_id in the in-memory object for later steps
                updated_article_count += 1
            else:
                print(f"  Warning: Article '{article_number}->{article_db_id}' not found in DB for law '{law_name}'. Cannot update its metadata or link concepts/relationships.")
        print(f"  Finished updating metadata for {updated_article_count} articles.")


        # 5. Upsert legal concepts (shared table)
        for concept in lm.legal_concepts:
            concept_code = concept.get('代號')
            concept_name = concept.get('詞彙名稱')
            concept_data_json = json.dumps(concept)
            concept_upsert_query = """
            INSERT INTO legal_concepts (law_id, code, name, data) 
            VALUES (%s, %s, %s, %s) 
            ON CONFLICT (code) DO UPDATE SET name = EXCLUDED.name, data = EXCLUDED.data, law_id = EXCLUDED.law_id
            RETURNING id;
            """
            concept_id_result = fetch_query(conn, concept_upsert_query, (law_db_id, concept_code, concept_name, concept_data_json))
            if concept_id_result and concept_id_result[0]:
                concept['db_id'] = concept_id_result[0][0]
                concept_code_to_db_id_map[concept_code] = concept['db_id']
            else:
                print(f"  Warning: Failed to upsert legal_concept {concept_code}.")

        # 6. Insert new law hierarchy relationships
        for relation in lm.hierarchy_relations:
            rel_code = relation.get('關係代號')
            main_law_name_hier = relation.get('主法規')
            related_law_name_hier = relation.get('關聯法規')
            
            main_law_pcode_hier = _get_law_pcode_by_name(main_law_name_hier, conn)
            related_law_pcode_hier = _get_law_pcode_by_name(related_law_name_hier, conn)

            hierarchy_type = relation.get('階層關係類型')
            relation_data_json = json.dumps(relation)
            
            main_law_db_id_hier = _get_law_db_id_by_pcode(main_law_pcode_hier, conn)
            related_law_db_id_hier = _get_law_db_id_by_pcode(related_law_pcode_hier, conn)

            if main_law_db_id_hier and related_law_db_id_hier and rel_code:
                hier_insert_query = """
                INSERT INTO law_hierarchy_relationships 
                    (relationship_code, main_law_id, related_law_id, hierarchy_type, data)
                VALUES (%s, %s, %s, %s, %s)
                ON CONFLICT (relationship_code) DO NOTHING;
                """
                execute_query(conn, hier_insert_query, (rel_code, main_law_db_id_hier, related_law_db_id_hier, hierarchy_type, relation_data_json))
            else:
                print(f"  Warning: Skipping law_hierarchy_relationship for {rel_code} due to missing law pcode/IDs or relation code. Main Law: '{main_law_name_hier}' (found pcode: {main_law_pcode_hier}), Related Law: '{related_law_name_hier}' (found pcode: {related_law_pcode_hier}).")

        # 7. Insert new law relationships
        for relation in lm.law_relations:
            rel_code = relation.get('代號')
            main_law_pcode_rel = relation.get('主法規代號')
            main_article_num_rel = relation.get('主法條條號')
            related_law_pcode_rel = relation.get('關聯法規代號')
            related_article_num_rel = relation.get('關聯法條條號')
            relationship_type = relation.get('關聯類型', relation.get('relationship_type'))
            relation_data_json = json.dumps(relation)
            
            main_law_db_id_rel = _get_law_db_id_by_pcode(main_law_pcode_rel, conn)
            related_law_db_id_rel = _get_law_db_id_by_pcode(related_law_pcode_rel, conn)
            
            main_article_db_id_rel = None
            if main_law_db_id_rel and main_article_num_rel:
                main_article_db_id_rel = _get_article_db_id_by_law_db_id_and_article_number(main_law_db_id_rel, main_article_num_rel, conn)
            
            related_article_db_id_rel = None
            if related_law_db_id_rel and related_article_num_rel:
                related_article_db_id_rel = _get_article_db_id_by_law_db_id_and_article_number(related_law_db_id_rel, related_article_num_rel, conn)
            
            if rel_code and relationship_type:
                law_rel_insert_query = """
                INSERT INTO law_relationships 
                    (code, relationship_type, main_law_id, main_article_id, related_law_id, related_article_id, data)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (code) DO NOTHING;
                """
                execute_query(conn, law_rel_insert_query, (
                    rel_code, relationship_type, 
                    main_law_db_id_rel, main_article_db_id_rel,
                    related_law_db_id_rel, related_article_db_id_rel,
                    relation_data_json
                ))
            else:
                print(f"  Warning: Skipping law_relationship for {rel_code} due to missing code or type.")

        # 8. Populate article_legal_concept table
        for article in lm.law_articles:
            article_db_id = article.get('db_id') # This now depends on the successful update in step 4
            if not article_db_id:
                continue
            related_concept_codes = article.get('相關概念代號列表', [])
            for concept_code in related_concept_codes:
                concept_db_id = concept_code_to_db_id_map.get(concept_code)
                if concept_db_id:
                    alc_insert_query = """
                    INSERT INTO article_legal_concept (article_id, legal_concept_id)
                    VALUES (%s, %s) ON CONFLICT DO NOTHING;
                    """
                    execute_query(conn, alc_insert_query, (article_db_id, concept_db_id))
                else:
                    print(f"    Warning: Concept DB ID not found for code {concept_code} when linking to article {article.get('條號')}.")
        
        conn.commit()
        print(f"Upsert process for law pcode {pcode} completed successfully.")
        return True
    except Exception as e:
        print(f"An overall error occurred during upsert_law_metadata_to_db for {pcode}: {e}")
        if conn: conn.rollback()
        return False



### B. Creating the Database Schema
This step is crucial and should be performed once to set up the necessary tables in your PostgreSQL database. It executes the SQL commands defined in `law_db_law.sql`.
**Prerequisite:**
- Database connection parameters must be correctly set in the utility cell above.
- The `law_db_law.sql` file must be present in the same directory as this notebook.

In [ ]:
print("--- B. Creating Database Schema ---")
conn_schema = get_db_connection()
if conn_schema:
    try:
        if 0:
            create_db_schema(conn_schema)
    finally:
        conn_schema.close()
        print("DB connection for schema creation closed.")
else:
    print("Failed to connect to the database. Schema creation skipped.")

### C. Loading JSON Data and Upserting to Database
This section demonstrates how to load `LawMetadata` from JSON files and then save it to the database.

#### C.ii. Upserting Multiple `LawMetadata` Objects via `LawMetadataMgr` (from JSON files to DB)
This demonstrates using `LawMetadataMgr` to load multiple laws from JSON files. If the manager is initialized with a database connection, laws are automatically upserted to the DB when added to the manager via `add_lm` (which is called by `load_lm_bynames_from_json`).
**Prerequisites:**
- JSON files for the specified laws (e.g., "政府採購法", "民法") must exist.
- Database schema must be created.

#### Issues
- 目前有的 json 約需跑 20 分鐘，還有些 warning, error 需要 fix

In [ ]:
print("--- C.ii. Upserting Multiple Laws via LawMetadataMgr (JSON to DB) ---")

# This cell uses LawMetadataMgr to load laws from JSON and upsert them to the DB.
# It supports multiple modes for selecting which laws to process.
# The logic correctly uses the file's "short name" to locate JSON files,
# and then uses the true "法規名稱" from the JSON content for all database interactions.

# --- Configuration for Upsert ---
# Set the desired upsert mode. Options are:
# "single": Upsert one specific law defined in `law_to_upsert_single`.
# "multiple": Upsert a list of laws defined in `laws_to_upsert_multiple`.
# "all_from_json": Upsert all laws that have corresponding JSON files in the `dir_json` directory.
upsert_mode = "single"  # <-- CHANGE THIS to "single" or "multiple", "all_from_json" as needed

# --- Define laws to be upserted based on their short name (filename) ---
law_to_upsert_single = "預算法"  # Used when upsert_mode is "single"
laws_to_upsert_multiple = ["政府採購法", "民法"]  # Used when upsert_mode is "multiple"

# --- Main Upsert Logic ---
available_json_laws = get_law_names_from_directory(dir_json)
print(f"Available JSON laws (by short name) in '{dir_json}': {available_json_laws}")

short_names_to_upsert = []
if upsert_mode == "single":
    if law_to_upsert_single and law_to_upsert_single in available_json_laws:
        short_names_to_upsert.append(law_to_upsert_single)
    elif not law_to_upsert_single:
         print("Warning: Upsert mode is 'single' but no law name is specified in 'law_to_upsert_single'.")
    else:
         print(f"Warning: Law '{law_to_upsert_single}' not found in available JSON files.")
elif upsert_mode == "multiple":
    short_names_to_upsert.extend([name for name in laws_to_upsert_multiple if name in available_json_laws])
    if len(short_names_to_upsert) != len(laws_to_upsert_multiple):
        print(f"Warning: Some laws in 'laws_to_upsert_multiple' were not found in the available JSON files.")
elif upsert_mode == "all_from_json":
    short_names_to_upsert.extend(available_json_laws)
    print(f"Found {len(available_json_laws)} laws to potentially upsert from the JSON directory.")
else:
    print(f"Error: Invalid upsert_mode '{upsert_mode}'. Please choose 'single', 'multiple', or 'all_from_json'.")

if not short_names_to_upsert:
    print("No laws to upsert. Exiting.")
else:
    print(f"Will attempt to load and upsert the following laws (by short name): {short_names_to_upsert}")
    conn_mgr_upsert = get_db_connection()
    if conn_mgr_upsert:
        # Initialize the manager with the DB connection.
        lmmgr_for_multi_upsert = LawMetadataMgr(db_conn=conn_mgr_upsert)
        try:
            # This method iterates through the short names, loads the corresponding files,
            # creates LawMetadata objects (which contain the *real* law name), and then
            # the manager's add_lm method calls the upsert function for each.
            lmmgr_for_multi_upsert.load_lm_bynames_from_json(short_names_to_upsert)
            
            print(f"\n--- Multi-Upsert Summary ---")
            print(f"Finished processing {len(short_names_to_upsert)} laws for upsert via LawMetadataMgr.")
            print(f"Laws now in manager (by real name): {list(lmmgr_for_multi_upsert.lms.keys())}")
        except Exception as e_mgr_upsert:
            print(f"An error occurred during multi-upsert via LawMetadataMgr: {e_mgr_upsert}")
        finally:
            conn_mgr_upsert.close()
            print("\nDB connection for multi-upsert via LawMetadataMgr closed.")
    else:
        print("Failed to connect to DB for multi-law upsert via LawMetadataMgr.")

### D. Loading `LawMetadata` from Database using `LawMetadataMgr`
These examples show how to use `LawMetadataMgr` (initialized with a database connection) to load `LawMetadata` objects directly from the PostgreSQL database. When `LawMetadataMgr` is initialized with a `db_conn`, its methods `add_lm` and `remove_lm` interact with the database. The `load_lm_from_db` method specifically fetches data from the database to populate the manager's in-memory collection.
**Prerequisites:**
- Database schema must be created (Section B).
- Data for the laws to be loaded must exist in the database (e.g., from Section C).

In [ ]:
print("--- D.i. Loading a Specific Law from DB into LawMetadataMgr ---")
conn_load_single_db = get_db_connection()
if conn_load_single_db:
    lmmgr_load_single = LawMetadataMgr(db_conn=conn_load_single_db)
    try:
        law_code_to_load_specific = example_law_code_for_later_use if 'example_law_code_for_later_use' in locals() and example_law_code_for_later_use else "LT_政府採購法"
        law_name_to_find_specific = example_law_name_for_later_use if 'example_law_name_for_later_use' in locals() and example_law_name_for_later_use else "政府採購法"

        print(f"Attempting to load law with code '{law_code_to_load_specific}' from DB...")
        lmmgr_load_single.load_lm_from_db(law_pcode_to_load=law_code_to_load_specific) #param name updated for clarity
        
        loaded_lm_specific = lmmgr_load_single.find_lm(law_name_to_find_specific)
        if loaded_lm_specific:
            print(f"Successfully loaded and found '{loaded_lm_specific.law_name}' (Code: {loaded_lm_specific.law_regulation.get('代號')}) from DB.")
            print(f"  Number of articles: {len(loaded_lm_specific.law_articles)}")
            print(f"  Number of concepts: {len(loaded_lm_specific.legal_concepts)}")
        else:
            print(f"Could not find law with name '{law_name_to_find_specific}' (tried code '{law_code_to_load_specific}') in manager after DB load attempt.")
    finally:
        conn_load_single_db.close()
        print("DB connection for loading single law closed.")
else:
    print("Failed to connect to DB for loading a single law.")

In [ ]:
print("\n--- D.ii. Loading All Laws from DB into LawMetadataMgr ---")
conn_load_all_db = get_db_connection()
if conn_load_all_db:
    lmmgr_load_all = LawMetadataMgr(db_conn=conn_load_all_db)
    try:
        print("Attempting to load ALL laws from DB...")
        lmmgr_load_all.load_lm_from_db() # Load all
        print(f"Total laws in manager after loading all from DB: {len(lmmgr_load_all.lms)}")
        if lmmgr_load_all.lms:
            print("Listing loaded laws:")
            for law_obj_name in lmmgr_load_all.lms.keys():
                print(f"- {law_obj_name}")
        else:
            print("No laws were loaded from the database.")
    finally:
        conn_load_all_db.close()
        print("DB connection for loading all laws closed.")
else:
    print("Failed to connect to DB for loading all laws.")

### E. Deleting `LawMetadata` from Database
This demonstrates how to delete a specific law and its associated data from the database using the standalone `delete_law_metadata_from_db` function. Deletion can also occur via `LawMetadataMgr.remove_lm()` if the manager is DB-connected.
**Prerequisites:**
- The law to be deleted must exist in the database.
- The `law_code` for the law must be known (e.g., from a previous upsert operation).

In [ ]:
print("--- E. Deleting Law(s) from DB ---")

# --- Configuration for Deletion ---
# Set the desired deletion mode. Options are:
# "single": Delete one specific law defined in `law_to_delete_single`.
# "multiple": Delete a list of laws defined in `laws_to_delete_multiple`.
# "all_from_json": Delete all laws that have corresponding JSON files in the `dir_json` directory.
delete_mode = "multiple" # <-- CHANGE THIS to "multiple" or "all_from_json" as needed

# --- Define laws to be deleted based on their short name (filename) ---
law_to_delete_single = "憲法增修合併" # Used when delete_mode is "single"
laws_to_delete_multiple = ['憲法合併', '憲法增修合併', '刑法'] # Used when delete_mode is "multiple"


# --- Main Deletion Logic ---
conn_delete = get_db_connection()
if conn_delete:
    try:
        short_names_to_delete = []
        if delete_mode == "single":
            if law_to_delete_single:
                short_names_to_delete.append(law_to_delete_single)
            else:
                print("Warning: Delete mode is 'single' but no law name is specified in 'law_to_delete_single'.")
        elif delete_mode == "multiple":
            short_names_to_delete.extend(laws_to_delete_multiple)
        elif delete_mode == "all_from_json":
            print(f"Getting all law names from directory: {dir_json}")
            json_law_names = get_law_names_from_directory(dir_json)
            short_names_to_delete.extend(json_law_names)
            print(f"Found {len(json_law_names)} laws to potentially delete.")
        else:
            print(f"Error: Invalid delete_mode '{delete_mode}'. Please choose 'single', 'multiple', or 'all_from_json'.")

        if not short_names_to_delete:
            print("No laws to delete. Exiting.")
        else:
            print(f"Attempting to delete the following laws (by short name): {short_names_to_delete}")
            deleted_count = 0
            not_found_count = 0
            for short_name in short_names_to_delete:
                print(f"--- Processing '{short_name}' for deletion ---")
                
                # First, read the regulation file to get the REAL law name.
                reg_filepath = os.path.join(dir_json, f"{short_name}_law_regulation.json")
                reg_data = load_json_data(reg_filepath)

                if not reg_data:
                    print(f"  Warning: Could not find or load '{reg_filepath}'. Cannot determine real law name. Skipping.")
                    not_found_count += 1
                    continue
                
                real_law_name = reg_data.get("法規名稱")
                if not real_law_name:
                    print(f"  Warning: '法規名稱' key not found in '{reg_filepath}'. Skipping.")
                    not_found_count += 1
                    continue

                # Find the pcode for the law using its REAL name.
                print(f"  Real name is '{real_law_name}'. Finding its pcode in the database...")
                pcode_to_delete = _get_law_pcode_by_name(real_law_name, conn_delete)

                if pcode_to_delete:
                    print(f"  Found pcode '{pcode_to_delete}'. Proceeding with deletion.")
                    # The delete function uses pcode, and ON DELETE CASCADE in the DB handles the rest.
                    success = delete_law_metadata_from_db(pcode_to_delete, conn_delete)
                    if success:
                        deleted_count += 1
                else:
                    print(f"  Warning: Law with real name '{real_law_name}' not found in the database. Cannot delete.")
                    not_found_count += 1
            
            print("\n--- Deletion Summary ---")
            print(f"Successfully deleted: {deleted_count} law(s).")
            print(f"Skipped (not found or file error): {not_found_count} law(s).")

    finally:
        conn_delete.close()
        print("\nDB connection for deletion closed.")
else:
    print("Failed to connect to the database. Deletion skipped.")

# 進階搜尋與分析 (Analysis Tools)
This section defines and demonstrates various analysis tools that can operate on `LawMetadata` objects, whether they are loaded from JSON files or from the database.

In [ ]:
# Analysis function definitions (keyword_search, category_filter, etc.)
def keyword_search(metadata, keyword, fields_to_search):
    """
    在指定的 MetaData 欄位中搜尋關鍵字。
    Args:
        metadata (LawMetaData): LawMetaData 物件。
        keyword (str): 要搜尋的關鍵字。
        fields_to_search (list of str): 要搜尋的欄位名稱列表 (例如 ['簡述', '定義'])。
    Returns:
        list: 包含符合條件的 MetaData 物件 (或其子物件)。
    """
    results = []
    if not metadata: return results # Guard against None metadata
    if "簡述" in fields_to_search and hasattr(metadata, 'law_regulation') and metadata.law_regulation:
        if keyword.lower() in metadata.law_regulation.get("簡述", "").lower():
            results.append(metadata.law_regulation)
    if "定義" in fields_to_search and hasattr(metadata, 'legal_concepts') and metadata.legal_concepts:
        for concept in metadata.legal_concepts:
            if keyword.lower() in concept.get("定義", "").lower():
                results.append(concept)
    return results

def category_filter(metadata, category_type, metadata_type="hierarchy_relations"):
    """
    篩選特定類型的 MetaData。
    Args:
        metadata (LawMetaData): LawMetaData 物件。
        category_type (str or Enum): 要篩選的類別/類型 (例如 "子法規", ConceptCategory.CORE_CONCEPT_DEFINITION)。
        metadata_type (str, optional): 要篩選的 MetaData 類型 (預設為 "hierarchy_relations")。
    Returns:
        list: 符合類別/類型的 MetaData 物件列表。
    """
    results = []
    if not metadata: return results # Guard against None metadata
    data_list = getattr(metadata, metadata_type, []) 
    for item in data_list:
        if item.get("階層關係類型") == category_type or \
           (isinstance(category_type, Enum) and item.get("概念類別") == category_type.value) or \
           (not isinstance(category_type, Enum) and item.get("概念類別") == category_type):
            results.append(item)
    return results

def combined_search_hierarchy(metadata, category_type, related_law_keyword):
    """
    複合條件搜尋法規階層關係 (特定類型 + 關聯法規關鍵字)。
    Args:
        metadata (LawMetaData): LawMetaData 物件。
        category_type (str): 要篩選的階層關係類型 (例如 "子法規")。
        related_law_keyword (str): 關聯法規名稱中要包含的關鍵字 (例如 "施行細則")。
    Returns:
        list: 符合複合條件的法規階層關係 MetaData 物件列表。
    """
    results = []
    if not metadata: return results # Guard against None metadata
    for relation in metadata.hierarchy_relations:
        if relation.get("階層關係類型") == category_type and related_law_keyword.lower() in relation.get("關聯法規", "").lower():
            results.append(relation)
    return results

def generate_mermaid_diagram(metadata):
    """
    生成法規關係圖的 mermaid 語法。
    Args:
        metadata (LawMetaData): LawMetaData 物件。
    Returns:
        str: mermaid 語法字串。
    """
    mermaid_lines = []
    if not metadata: return "graph TD\n    %% No metadata provided"
    for relation in metadata.hierarchy_relations:
        main_law = relation.get("主法規", "N/A")
        related_law = relation.get("關聯法規", "N/A")
        relation_type = relation.get("階層關係類型", "related")
        mermaid_lines.append(f'    {main_law} -->|{relation_type}| {related_law}')
    if not mermaid_lines:
        return "graph TD\n    %% No hierarchy relations found"
    return "graph TD\n" + "\n".join(mermaid_lines)

def generate_article_mermaid_diagram(metadata):
    """
    生成法條關係圖的 mermaid 語法。
    Args:
        metadata (LawMetaData): LawMetaData 物件。
    Returns:
        str: mermaid 語法字串。
    """
    mermaid_lines = ["graph TD"]
    if not metadata: return "graph TD\n    %% No metadata provided"
    for article in metadata.law_articles:
        article_id = article.get("條號", "UnknownArticle")
        related_articles_map = article.get("法條關聯性", {}) 
        for rel_type, related_items in related_articles_map.items():
            if related_items and isinstance(related_items, list):
                for v_item in related_items:
                    v_clean = str(v_item).replace("、","_").replace("，","_").replace(" ","_").replace("『","_").replace("』","_")
                    mermaid_lines.append(f'    A_{article_id}["{article_id}"] --> |{rel_type}| R_{v_clean}["{v_item}"]')
    if len(mermaid_lines) == 1:
        return "graph TD\n    %% No article relations found"
    return "\n".join(mermaid_lines)

print("Analysis tool functions defined.")

### F. Analyzing DB-Loaded Data
This section demonstrates using the analysis tools defined above with `LawMetadata` objects loaded from the PostgreSQL database.
**Prerequisite:** Ensure that a law (e.g., "政府採購法" with its corresponding code) has been successfully upserted into the database.

In [ ]:
print("--- F. Testing Analysis Tools with DB-Loaded LawMetadata ---")
conn_analysis_db = get_db_connection()
lm_db_for_analysis = None

if conn_analysis_db:
    try:
        lmmgr_db_analysis_tools = LawMetadataMgr(db_conn=conn_analysis_db)
        
        law_code_for_analysis = example_law_code_for_later_use if 'example_law_code_for_later_use' in locals() and example_law_code_for_later_use else "LT_政府採購法"
        law_name_for_analysis = example_law_name_for_later_use if 'example_law_name_for_later_use' in locals() and example_law_name_for_later_use else "政府採購法"
        
        print(f"Attempting to load '{law_name_for_analysis}' (code: {law_code_for_analysis}) from DB for analysis...")
        lmmgr_db_analysis_tools.load_lm_from_db(law_pcode_to_load=law_code_for_analysis) #param name updated
        lm_db_for_analysis = lmmgr_db_analysis_tools.find_lm(law_name_for_analysis)

        if lm_db_for_analysis:
            print(f"\nSuccessfully loaded '{law_name_for_analysis}' from DB for analysis.")
            
            print(f"\n--- Keyword Search on DB-loaded '{law_name_for_analysis}' (Keyword: '採購') ---")
            db_keyword_results = keyword_search(lm_db_for_analysis, "採購", ["簡述", "定義"])
            if db_keyword_results:
                for res in db_keyword_results:
                    print(f"  - Found in: {res.get('詞彙名稱', res.get('法規名稱', 'N/A'))}, Content: {res.get('定義', res.get('簡述', ''))[:100]}...")
            else:
                print("  No results for keyword search.")

            print(f"\n--- Category Filter on DB-loaded '{law_name_for_analysis}' (Category: '子法規') ---")
            db_category_results = category_filter(lm_db_for_analysis, "子法規")
            if db_category_results:
                for res in db_category_results:
                    print(f"  - Main: {res.get('主法規')}, Related: {res.get('關聯法規')}, Type: {res.get('階層關係類型')}")
            else:
                print("  No results for category filter '子法規'.")
            
            print(f"\n--- Mermaid Diagram for DB-loaded '{law_name_for_analysis}' (Hierarchy) ---")
            db_mermaid_output = generate_mermaid_diagram(lm_db_for_analysis)
            print(db_mermaid_output)
        else:
            print(f"Failed to load or find '{law_name_for_analysis}' from DB for analysis. Ensure it was upserted with code '{law_code_for_analysis}'.")

    except Exception as e_analysis_db:
        print(f"An error occurred during DB analysis demonstration: {e_analysis_db}")
    finally:
        if conn_analysis_db and not conn_analysis_db.closed:
            conn_analysis_db.close()
            print("\nDB connection for analysis closed.")
else:
    print("Failed to connect to DB for analysis demonstration.")


In [ ]:
if 0: # Original examples with JSON-loaded data (lmmgr_json_ops)
    print("\n--- Testing Analysis Tools with JSON-Loaded LawMetadata (from lmmgr_json_ops) ---")
    # This cell is for comparison, using LawMetadata loaded from JSON files via lmmgr_json_ops.
    # Ensure lmmgr_json_ops is initialized and populated (e.g., from cell under '法規管理使用' section if it was run).
    if 'lmmgr_json_ops' in locals() and lmmgr_json_ops.lms and len(lmmgr_json_ops.lms) > 0:
        first_json_law_name = list(lmmgr_json_ops.lms.keys())[0]
        lm_json_test = lmmgr_json_ops.find_lm(first_json_law_name)
        
        if lm_json_test:
            print(f"Using '{lm_json_test.law_name}' loaded from JSON for comparison.")
            print(f"\n--- Keyword Search on JSON-loaded '{lm_json_test.law_name}' (Keyword: '公開') ---")
            json_keyword_results = keyword_search(lm_json_test, "公開", ["簡述", "定義"])
            if json_keyword_results:
                for res in json_keyword_results:
                    print(f"  - Found in: {res.get('詞彙名稱', res.get('法規名稱', 'N/A'))}, Content: {res.get('定義', res.get('簡述', ''))[:100]}...")
            else:
                print("  No results for keyword search.")

            print(f"\n--- Category Filter on JSON-loaded '{lm_json_test.law_name}' (Category: '子法規') ---")
            json_category_results = category_filter(lm_json_test, "子法規")
            if json_category_results:
                for res in json_category_results:
                    print(f"  - Main: {res.get('主法規')}, Related: {res.get('關聯法規')}, Type: {res.get('階層關係類型')}")
            else:
                print("  No results for category filter '子法規'.")
            
            print(f"\n--- Mermaid Diagram for JSON-loaded '{lm_json_test.law_name}' (Hierarchy) ---")
            json_mermaid_output = generate_mermaid_diagram(lm_json_test)
            print(json_mermaid_output)
        else:
            print(f"Could not find law '{first_json_law_name}' in lmmgr_json_ops to run JSON-based analysis tests.")
    else:
        print("LawMetadata manager for JSON (lmmgr_json_ops) not populated or empty. Skipping JSON-based analysis tests.")

# 單法規使用

In [ ]:
law_name = "政府採購法施行細則" # Or dynamically extract from law_regulation.json if needed

filepaths = {
    "law_regulation": f"{dir_json}/{law_name}_law_regulation.json",
    "legal_concepts": f"{dir_json}/{law_name}_legal_concepts.json",
    "hierarchy_relations": f"{dir_json}/{law_name}_hierarchy_relations.json",
    "law_relations": f"{dir_json}/{law_name}_law_relations.json",
    "law_articles": f"{dir_json}/{law_name}_law_articles.json"
}


# 1. Read from separate JSON files and create LawMetadata object
lm = LawMetadata.from_json_files(**filepaths)


if lm:
    print(lm)

    # 2. Access Metadata content (example)
    print("\nLaw Regulation Name:", lm.law_regulation.get("法規名稱"))
    print("\nFirst Legal Concept Name:", lm.legal_concepts[0].get("詞彙名稱"))

    if 0:
        lm.renew_id()
    # 3. Modify Metadata content (example)
    if 0:
        lm.law_regulation["版本"] = "20250312-Test Version"
        new_concept = {
            "代號": "concept-gpa-new-concept",
            "詞彙名稱": "New Concept",
            "定義": "This is a new concept definition.",
            "相關概念": [],
            "相關法條": [],
            "概念類別": "新增概念",
            "同義詞": [],
            "台灣觀點": "Taiwan Viewpoint.",
            "範例": "Example here.",
            "語意向量": "[]"
        }
        lm.legal_concepts.append(new_concept)
if 0:
    # 4. Export LawMetadata object to separate JSON files
    lm.to_json_files(output_prefix=f"{dir_json}/{law_name}_M") # Exports to gpa_modified_*.json
if 1:
    conn = get_db_connection()
    upsert_law_metadata_to_db(lm,conn)

# 法規管理使用

In [ ]:
if 0: #法律管理
    lmmgr = LawMetadataMgr()
    law_names = get_law_names_from_directory("./json")
    #print(law_names)
    #lmmgr.load_lm_bynames(["憲法合併","憲法增修合併","刑法","民法","行政程序法","預算法","政府採購法","政府採購法施行細則"])
    lmmgr.load_lm_bynames_from_json(law_names)   
    lms = lmmgr.lms
if 0: #法律列表
    for law_name in lms.keys():
        #print(f"{lms[law_name].law_name}-{lms[law_name].short_name}" )
        print(f"{lms[law_name].law_name}" )
if 0: #法律概念列表
    for law_name in lms.keys():
        print(f"----- {law_name} -----")
        for lc in lms[law_name].legal_concepts:
            print(f"{lc['詞彙名稱']}")
    #print(lms['中華民國憲法(合併增修條文)'].legal_concepts)
if 0: # 某法跟什麼法有關係如 公司法
    target = "公司法"
    for law_name in lms.keys():
        if target in lms[law_name].law_regulation['相關法規']:
            print(f"{law_name} 跟 {target} 有關係")
if 1: # 某個關鍵字的總覽
   
    pass
 

## LLM 建構 json
- 由於 LLM 的產出有時遇到小問題，所以使用時暫時需開開關關

In [ ]:
import base64
import os
# from google import genai # Assuming genai might not be available in all test environments
# from google.genai import types
import time

def generate(client,files,law_name,user_prompt,file_path):
    # This function is a placeholder if genai is not available.
    # To use it, ensure google.genai is installed and configured.
    print(f"[INFO] Generate function called for {law_name} with prompt: {user_prompt[:50]}...")
    print(f"[INFO] Output would be written to {file_path}")
    # Actual generation logic commented out for environments without genai
    """
    model = "gemini-2.0-flash-thinking-exp-01-21"
    if 1:
        contents = [
            types.Content(
                role="user",
                parts=[
                    types.Part.from_uri(
                        file_uri=files[0].uri,
                        mime_type=files[0].mime_type,
                    ),
                    types.Part.from_uri(
                        file_uri=files[1].uri,
                        mime_type=files[1].mime_type,
                    ),
                    types.Part.from_text(text=user_prompt),
                ],
            )
        ]
        generate_content_config = types.GenerateContentConfig(
            temperature=0.7,
            top_p=0.95,
            top_k=64,
            max_output_tokens=65536,
            response_mime_type="text/plain",
            system_instruction=[
                types.Part.from_text(text="""請以台灣人的立場，用繁體中文回答"""),
            ],
        )
        
        print(f"Q::{user_prompt}")

        response_text = ""
        for chunk in client.models.generate_content_stream(
            model=model,
            contents=contents,
            config=generate_content_config,
        ):
            response_text += chunk.text
            #print(chunk.text, end="")
        print(f"A::{response_text}")

        try:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(f"Q::{user_prompt}\n")
                file.write(f"A::{response_text}")
                print(f"Content written to {file_path}")
        except Exception as e:
            print(f"An error occurred: {e}")
    """
    pass # Placeholder if genai is not used

api_key=os.environ.get("GEMINI_API_KEY")
client = None # Placeholder for genai.Client
files_for_llm = [] # Placeholder for uploaded files

law_name="憲法增修合併" # Example law name for LLM section

if 0: # This entire LLM section is turned off by default for this refactoring step
    # Ensure google.genai is imported and client is initialized if this block is enabled
    # client = genai.Client(api_key=api_key)
    # files_for_llm = [
    #     client.files.upload(file=f"{law_name}.md",config={'mime_type':"text/markdown"}),
    #     client.files.upload(file="法律語法形式化.md",config={'mime_type':"text/markdown"}),
    # ]
    pass

if 0: # 產生非法條的 Meta data (Turned off)
    prompt_list=[
        ['law_regulation',f"""根據{law_name}的整體資訊，按照法律語法形式化的設計，依照裡面範例格式，產生法規 Meta Data,盡可能詳列資訊，不要省略"""],
        ['legal_concepts',f"""根據{law_name}的整體資訊，按照法律語法形式化的設計，依照裡面範例格式，產生法律概念 Meta Data (Legal Concept Meta Data)，注意並非 法規 Meta Data，請列出全部概念，不要省略"""],
        ['hierarchy_relations',f"""根據{law_name}的整體資訊，按照法律語法形式化的設計，依照裡面範例格式，產生法規階層關係 Meta Data，不包含法條間關聯性，請列出全部法規間階層關係，尤其包含上位關係，不要省略"""],
        ['law_relations',f"""根據{law_name}的整體資訊，按照法律語法形式化的設計，依照裡面範例格式，產生法規關聯性 Meta Data，不包含本法規內部法條之間的關聯性，請列出法規間全部關聯性，不要省略"""]
    ]
    for prompt_pair in prompt_list:
        file_path_txt = f"{dir_txt}/{law_name}_{prompt_pair[0]}.txt"
        if client and files_for_llm: generate(client,files_for_llm,law_name,prompt_pair[1],file_path_txt)
        
        file_path_json = f"{dir_json}/{law_name}_{prompt_pair[0]}.json"
        if os.path.exists(file_path_txt):
            regex = r"```(?:json)*\n(.*)```"
            lines = handle_regex(regex,file_path_txt,"col1")
            if lines:
                json_string = "\n".join(lines)
                json_string = re.sub(r'//.*', '', json_string)
                json_string = re.sub(r'# .*', '', json_string)
                try:
                    json_object = json.loads(json_string)
                    print(json_object)
                    with open(file_path_json, 'w', encoding='utf-8') as f:
                        json.dump(json_object, f, indent=2, ensure_ascii=False)
                except json.JSONDecodeError as je:
                    print(f"JSON Decode Error for {file_path_txt}: {je}")
            else:
                print(f"No JSON content found in {file_path_txt}")

if 0: # 取得最大條號 ，也可手動填入結果 (Turned off)
    # ... (LLM code remains off for this refactoring)
    pass
    
if 0: # 產生法條 Meta Data (Turned off)
    # ... (LLM code remains off for this refactoring)
    pass

if 0: #從檔案內組合法條 Meta data (Turned off)
    # ... (LLM code remains off for this refactoring)
    pass